In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale 
from sklearn import model_selection

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. (a) linear model validation error

In [14]:
# preprocessing
df0 = pd.read_csv("College.csv", header = 0)
df0.replace({'Yes':1,'No':0},inplace = True)
df1 = df0.iloc[:,1:]

X = df1.drop(['Apps'], axis = 'columns')
y = df1.Apps


In [15]:
#random split, 스케일조정하고 lr
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, random_state=3 )
lr1 = LinearRegression()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
lr1.fit(X_train_scaled, y_train)
y_pred1 = lr1.predict(scaler.transform(X_test))

#print("coefficient: ",lr1.coef_)
#print("intercept: ", lr1.intercept_)
print("test error: ", mean_squared_error(y_test, y_pred1))
print("r2: ", r2_score(y_test, y_pred1))

test error:  549860.2061479541
r2:  0.9282672082562653


## (b) ridge model

In [16]:
#lamda도출하기
ridge = Ridge()
kf = KFold(n_splits=10, shuffle=True, random_state=3)
alphas = 10**np.linspace(10,-2,100)*0.5
ridgecv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', cv=kf)
ridgecv.fit(X_train_scaled, y_train)
alpha_r = ridgecv.alpha_
print('alpha_r: ', alpha_r) 

#test 기준 mse, # Ridge regression model with lamda 0.0005
ridge2 = Ridge(alpha=alpha_r)
ridge2.fit(X_train_scaled, y_train)
print("test mse: ", mean_squared_error(y_test, ridge2.predict(scaler.transform(X_test))))
print("ridge r2: ", r2_score(y_test, ridge2.predict(scaler.transform(X_test))))

alpha_r:  0.005
test mse:  549845.7881064347
ridge r2:  0.9282690891822863


## c) lasso model

In [17]:
#Lasso alpha구하기
lasso = LassoCV(alphas = None, cv=kf, max_iter=10000)
lasso.fit(X_train_scaled, y_train)
print("alpha: ",lasso.alpha_)

#Lasso test 기준 mse:
lasso2 = Lasso(alpha=lasso.alpha_)
lasso2.fit(X_train_scaled, y_train)
print("test mse: ", mean_squared_error(y_test, lasso2.predict(scaler.transform(X_test))))
print("lasso r2: ", r2_score(y_test, lasso2.predict(scaler.transform(X_test))))

alpha:  13.137930371924996
test mse:  511394.442258026
lasso r2:  0.9332853139484555


In [7]:
#coefficient
for i in range(X.shape[1]):
    print('\"'+X.columns[i]+'\"', ' coefficient: ', lasso2.coef_[i])
    
print("사용된 feature수 : 17개 중 14개, \n제외된 feature 3개 : ", X.columns[5], X.columns[9], X.columns[14])

"Private"  coefficient:  -229.32887366792755
"Accept"  coefficient:  3783.8509827397247
"Enroll"  coefficient:  -365.7176385373201
"Top10perc"  coefficient:  747.4370748523329
"Top25perc"  coefficient:  -159.17336088258708
"F.Undergrad"  coefficient:  0.0
"P.Undergrad"  coefficient:  64.77443069369855
"Outstate"  coefficient:  -282.756492085014
"Room.Board"  coefficient:  154.79487644022532
"Books"  coefficient:  -0.0
"Personal"  coefficient:  13.915622898974275
"PhD"  coefficient:  -119.01961539102203
"Terminal"  coefficient:  -46.79427395477661
"S.F.Ratio"  coefficient:  43.48572275663173
"perc.alumni"  coefficient:  -0.0
"Expend"  coefficient:  385.0690736058035
"Grad.Rate"  coefficient:  128.78477745705808
사용된 feature수 : 17개 중 14개, 
제외된 feature 3개 :  F.Undergrad Books perc.alumni


## (d) comment

In [8]:
print("OLS의 test error: ", mean_squared_error(y_test, y_pred1))
print("ridge의 test error: ",  mean_squared_error(y_test, ridge2.predict(scaler.transform(X_test))))
print("lasso의 test error: ",  mean_squared_error(y_test, lasso2.predict(scaler.transform(X_test))))

OLS의 test error:  549860.2061479541
ridge의 test error:  549845.7881064347
lasso의 test error:  511394.442258026


# 2.Boston dataset

In [18]:
from sklearn.preprocessing import StandardScaler

from sklearn import model_selection

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SequentialFeatureSelector as sfs

In [19]:
# preprocessing
df0 = pd.read_csv("Boston.csv", header = 0)
df0.replace({'Yes':1,'No':0},inplace = True)
df1 = df0.iloc[:,1:]
df1.head

X = df1.drop(['crim'], axis = 'columns')
y = df1.crim

In [21]:
#전체 변수 활용시 bic, adjr
import statsmodels.api as stats
model0 = stats.OLS(y, X)
result0 = model0.fit()
print("bIC: ", result0.bic)
print("패키지adjr2: ", result0.rsquared_adj)

bIC:  3389.4220564895413
패키지adjr2:  0.5170355640341698


## (a) i) forward step-wise selection

In [22]:
###Forwardselection구현! with X전체 ,r2스코어로 > 변수다넣는게 좋게나옴
import statsmodels.api as stats

X_remain = X.copy()
lr1 = LinearRegression()
X_con = pd.DataFrame()
paramlist = []
bicDict = {}
adjrD={}

for j in range(X.shape[1]):

    if j==X.shape[1]-1:
        model = stats.OLS(y, X)
        result = model.fit()
        print("모든 변수 선택시: ")
        print("\t BIC: ", result.bic)
        bicDict[j] = result.bic
        print("\t 패키지adjr2: ", result.rsquared_adj)
        adjrD[j] = result.rsquared_adj
    #X1이 있는 상황을 가정하고, X2 두번째 forloop도는 경우    
    else:
        r2_dict = {}
        for i in range(X_remain.shape[1]):
            X_i = X_remain.iloc[:,[i]]
            lr1.fit(X_i, y)
            error = r2_score(y, lr1.predict(X_i))
            r2_dict[i]=error
        #X2_idx는 그냥 X_idx로 바꿔도 갠춘루프돌면서. 저장안하는변수
        Xj_idx = max(r2_dict, key=r2_dict.get)
        feature_names = np.array(X_remain.columns)
        paramlist.append(feature_names[Xj_idx])
        Xj= X_remain.iloc[:, [Xj_idx]]
        lr1.fit(Xj, y)
    #X_con = pd.concat([X1, X2], axis = 1) 이문장을 아래와같이 바꿈
        if (X_con.empty):
            X_con = Xj
        else:
            X_con = pd.concat([X_con,Xj], axis=1)

        X_remain= X_remain.drop(X_remain.columns[Xj_idx], axis = 1)

    #     print("j번째 X: ", Xj.head(3))
    #     print("j번째 합친 x:", X_con.head(3))
    #     print("j번째 X_remain: ", X_remain.head(3))
        lr1.fit(X_con, y)
        r2= r2_score(y, lr1.predict(X_con))
        print("변수", (j+1), "개 선택시 파라미터 list: ", paramlist)
        model = stats.OLS(y, X_con)
        result = model.fit()
        bic = result.bic
        bicDict[j]= bic
        print("\t BIC: ", bic)
        print("\t 패키지adjr2: ", result.rsquared_adj, "\n")
        adjrD[j] = result.rsquared_adj
print("가장 낮은 bic의 변수 개수: ", (min(bicDict, key=bicDict.get)+1))
print("가장 높은 adjr의 변수 개수: ", (max(adjrD, key=adjrD.get)+1))

변수 1 개 선택시 파라미터 list:  ['rad']
	 BIC:  3393.8220548446307
	 패키지adjr2:  0.45435136167438217 

변수 2 개 선택시 파라미터 list:  ['rad', 'tax']
	 BIC:  3381.5332400756433
	 패키지adjr2:  0.4729129017968581 

변수 3 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat']
	 BIC:  3367.705890258626
	 패키지adjr2:  0.49238681135644136 

변수 4 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 'nox']
	 BIC:  3363.9270870683586
	 패키지adjr2:  0.5013340815981708 

변수 5 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 'nox', 'indus']
	 BIC:  3370.0631618025363
	 패키지adjr2:  0.5004280611635488 

변수 6 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 'nox', 'indus', 'medv']
	 BIC:  3369.004328707831
	 패키지adjr2:  0.5065844852765944 

변수 7 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 'nox', 'indus', 'medv', 'dis']
	 BIC:  3371.444871461579
	 패키지adjr2:  0.5092811046065757 

변수 8 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 'nox', 'indus', 'medv', 'dis', 'age']
	 BIC:  3377.652866860061
	 패키지adjr2:  0.5083137424034628 

변수 9 개 선택시 파라미터 list:  ['rad', 'tax', 'lstat', 

## (a) ii) backward step-wise selection

In [23]:
X_remain = X.copy()
lr1 = LinearRegression()
X_con = pd.DataFrame()
unused_paramlist = []
bicDict2 = {}
adjrD2={}
for j in range(X.shape[1]-1):
    r2_dict = {}
    for i in range(X_remain.shape[1]):
#       대신  X_i = X_remain.iloc[:,[i]]
        X_remain_view= X_remain.drop(X_remain.columns[i], axis = 1)
        lr1.fit(X_remain_view, y)
        error = r2_score(y, lr1.predict(X_remain_view))
        r2_dict[i]=error
    #X2_idx는 그냥 X_idx로 바꿔도 갠춘루프돌면서. 저장안하는변수
    Xj_idx = max(r2_dict, key=r2_dict.get)
    feature_names = np.array(X_remain.columns)
    unused_paramlist.append(feature_names[Xj_idx])
    Xj= X_remain.iloc[:, [Xj_idx]]
    X_remain = X_remain.drop(X_remain.columns[Xj_idx], axis=1)
    lr1.fit(X_remain, y)
    print("변수", (j+1),"개 제외시 제거 변수 list: ", unused_paramlist)
    lr1.fit(X_remain, y)
    r2= r2_score(y, lr1.predict(X_remain))
    model = stats.OLS(y, X_remain)
    result = model.fit()
    print("\t BIC: ", result.bic)
    bicDict2[11-j] = result.bic
    adj_r2 = 1-((1-r2)*(X_remain.shape[0]-1)/(X_remain.shape[0]-(j+1)-1))
    print("\t 패키지adjr2: ", result.rsquared_adj)
    adjrD2[12-j] = result.rsquared_adj
print("가장 낮은 bic의 변수 개수: ", min(bicDict2, key=bicDict2.get))
print("가장 높은 adjr의 변수 개수: ", max(adjrD2, key=adjrD2.get))

변수 1 개 제외시 제거 변수 list:  ['age']
	 BIC:  3383.229381756349
	 패키지adjr2:  0.5179789936005856
변수 2 개 제외시 제거 변수 list:  ['age', 'indus']
	 BIC:  3377.71790881363
	 패키지adjr2:  0.5182705255756187
변수 3 개 제외시 제거 변수 list:  ['age', 'indus', 'chas']
	 BIC:  3372.1818642641347
	 패키지adjr2:  0.5185833026797249
변수 4 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 'tax']
	 BIC:  3366.6903634311125
	 패키지adjr2:  0.518851574835435
변수 5 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 'tax', 'rm']
	 BIC:  3366.721748108987
	 패키지adjr2:  0.5138402792161607
변수 6 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 'tax', 'rm', 'lstat']
	 BIC:  3367.6845541763937
	 패키지adjr2:  0.5078697594366834
변수 7 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 'tax', 'rm', 'lstat', 'ptratio']
	 BIC:  3363.403051743113
	 패키지adjr2:  0.5069604782907831
변수 8 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 'tax', 'rm', 'lstat', 'ptratio', 'nox']
	 BIC:  3365.3052833011916
	 패키지adjr2:  0.49997400988357177
변수 9 개 제외시 제거 변수 list:  ['age', 'indus', 'chas', 't

In [24]:
#bic기준 forward selection
lr2 = LinearRegression()
X_f = X[['rad', 'tax', 'lstat', 'nox']]
lr2.fit(X_f, y)
print('forward selection score: ', lr2.score(X_f, y))
print('coef: ', lr2.coef_)

forward selection score:  0.42157924340884856
coef:  [ 5.56194698e-01 -1.17537701e-03  2.56485639e-01 -2.50322274e+00]


In [25]:
#bic기준 backward selection
lr3 = LinearRegression()
X_b = X[['rad', 'medv']]
lr3.fit(X_b, y)
print('backward selection score: ', lr3.score(X_b, y)) 
print('coef: ', lr3.coef_)

backward selection score:  0.41745027938043744
coef:  [ 0.5519009  -0.16375784]


## (a) iii) ridge

In [26]:
X = df1.drop(['crim'], axis = 'columns')
y = df1.crim
##hyper params tuning
kf = KFold(n_splits=10, shuffle=True, random_state=5)
    #ridge
ridge3 = Ridge()
alphas = 10**np.linspace(10,-2,100)*0.5
parameters = {'alpha':alphas}
ridge_gr = GridSearchCV(ridge3, parameters, scoring='neg_mean_squared_error', cv=kf)
ridge_gr.fit(StandardScaler().fit_transform(X), y)
print(ridge_gr.best_params_)
ridge3 = Ridge(alpha= ridge_gr.best_params_['alpha'])
ridge3.fit(StandardScaler().fit_transform(X),y)
print("ridge coefficients: ", ridge3.coef_)
print("ridge score: ", ridge3.score(StandardScaler().fit_transform(X),y)) # r2기준 score

{'alpha': 5.361336110051605}
ridge coefficients:  [ 0.99046037 -0.46991963 -0.20675871 -1.01066293  0.4281705  -0.02588755
 -1.9878106   4.926048   -0.25443356 -0.5857582   1.0380787  -1.87844436]
ridge score:  0.44893471714113986


## (a) iv) lasso2

In [27]:
 #Lasso
lasso3 = LassoCV(alphas = None, cv=kf, max_iter=10000)
lasso3.fit(StandardScaler().fit_transform(X), y)
print("alpha: ",lasso3.alpha_)
lasso3 = Lasso(alpha=lasso3.alpha_)
lasso3.fit(StandardScaler().fit_transform(X),y)
for i in range(X.shape[1]):
    print('\"'+X.columns[i]+'\"', ' coefficient: ', lasso3.coef_[i])
    
print("사용된 feature수 : 11개, \n제외된 feature 1개 : ", X.columns[5])
print("lasso score: ", lasso3.score(StandardScaler().fit_transform(X),y))

alpha:  0.020236498255817244
"zn"  coefficient:  0.9854871505237517
"indus"  coefficient:  -0.43169196079975064
"chas"  coefficient:  -0.19267806700944312
"nox"  coefficient:  -1.0146885349453616
"rm"  coefficient:  0.379642153907219
"age"  coefficient:  -0.0
"dis"  coefficient:  -1.9513011159927467
"rad"  coefficient:  5.046752277243799
"tax"  coefficient:  -0.3404470944172445
"ptratio"  coefficient:  -0.5901919678107613
"lstat"  coefficient:  0.974945244560609
"medv"  coefficient:  -1.8706955079828145
사용된 feature수 : 11개, 
제외된 feature 1개 :  age
lasso score:  0.44899395161027744


## 2. (b) cross-validation

### b)i) linear regression

In [29]:
lr5 = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
score5 = cross_val_score(lr5, X, y, cv=kf, scoring='neg_mean_squared_error') 
mse5 = (-1)*score5.mean()
print('linear regression cv error: ', mse5)

linear regression cv error:  42.120328224886705


### b)ii) forward selection

In [30]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
X = df1.drop(['crim'], axis = 'columns')
y = df1.crim
error_df = pd.DataFrame()
n=0
mselist = []
lr = LinearRegression()
######forward selection using cross-validation
for train_idx, test_idx in kf.split(X):   
    X_train, X_test, y_train, y_test = X.loc[train_idx], X.loc[test_idx], y.loc[train_idx], y.loc[test_idx]
    for i in range(X.shape[1]-1):
        sfs1 = sfs(lr, direction='forward', n_features_to_select=i+1, cv=kf)
        sfs1.fit(X_train, y_train)
        #print(X.columns[sfs1.get_support()])
        X_train_sel= X_train.loc[:,sfs1.get_support()]
        X_test_sel = X_test.loc[:,sfs1.get_support()]
        #print(X_train_sel)
        lr.fit(X_train_sel, y_train)
        mse = mean_squared_error(y_test, lr.predict(X_test_sel))
        error_df.loc[n,i]=mse
    n+=1
#print(error_df)
cv_error = error_df.mean()
print("각 M(p)모델별 forward selection cv error: ", cv_error)
print("forward selection cv error: ", min(cv_error))

각 M(p)모델별 forward selection cv error:  0     45.289277
1     43.222396
2     43.161108
3     42.933356
4     43.043418
5     42.868096
6     42.678665
7     42.763557
8     42.542904
9     42.540238
10    42.499605
dtype: float64
forward selection cv error:  42.4996048111263


### b)iii) backward selection

In [31]:
X = df1.drop(['crim'], axis = 'columns')
y = df1.crim
error_df2 = pd.DataFrame()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
n=0
for train_idx, test_idx in kf.split(X):    
    X_train, X_test, y_train, y_test = X.loc[train_idx], X.loc[test_idx], y.loc[train_idx], y.loc[test_idx]
    for i in range(X.shape[1]-1):
        sfs1 = sfs(lr, direction='backward', n_features_to_select=i+1, cv=kf)
        sfs1.fit(X_train, y_train)
        
        X_train_sel= X_train.loc[:,sfs1.get_support()]
        X_test_sel = X_test.loc[:,sfs1.get_support()]
        lr.fit(X_train_sel, y_train)    
        #print(X.columns[sfs1.get_support()])
        mse = mean_squared_error(y_test, lr.predict(X_test_sel))
        error_df2.loc[n,i]=mse
    n+=1
cv_error2 = error_df2.mean()
print("각 M(p)모델별 forward selection cv error: ", cv_error2)
min(cv_error2)

각 M(p)모델별 forward selection cv error:  0     45.289277
1     43.353344
2     43.253285
3     42.808766
4     42.635412
5     42.538380
6     42.662366
7     42.767304
8     42.627428
9     42.633027
10    42.693356
dtype: float64


42.53838027244304

### b)iv)lasso

In [32]:
#data
X = df1.drop(['crim'], axis = 'columns')
y = df1.crim
kf = KFold(n_splits=5, shuffle=True, random_state=1)
X_scaled = StandardScaler().fit_transform(X) 


#Lasso alpha구하기
lass2 = LassoCV(alphas = None, cv=kf, max_iter=10000) 
lass2.fit(X_scaled, y)   
print("alpha: ",lass2.alpha_)
print("r2: ", lass2.score(X_scaled,y)) 

#Lasso: CV mse
Lasso3= Lasso(lass2.alpha_)
score3 = cross_val_score(Lasso3, X_scaled, y, cv=kf, scoring='neg_mean_squared_error') 
print(score3)
mse3 = (-1)*score3.mean()
print('cv error: ', mse3)

alpha:  0.05374991578689812
r2:  0.4472329074404988
[-16.0129356  -65.22224423 -63.63442149 -21.99876408 -43.25530316]
cv error:  42.02473371257594


### b)v)ridge

In [33]:
X = df1.drop(['crim'], axis = 'columns')
y = df1.crim
##hyper params tuning
kf = KFold(n_splits=5, shuffle=True, random_state=1)
    #ridge
ridg = Ridge()
alphas = 10**np.linspace(10,-4,100)*0.5
parameters = {'alpha':alphas}
ridge_gr = GridSearchCV(ridge2, parameters, scoring='neg_mean_squared_error', cv=kf)
ridge_gr.fit(StandardScaler().fit_transform(X), y)
print(ridge_gr.best_params_)
r_alpha=ridge_gr.best_params_['alpha']
ridge3 = Ridge(alpha= r_alpha)


#ridge mse
ridge3= Ridge(r_alpha)
score4 = cross_val_score(ridge3, X_scaled, y, cv=kf, scoring='neg_mean_squared_error') 
print(score4)
mse4 = (-1)*score4.mean()
print('ridge cv error: ', mse4)

{'alpha': 6.164233697210342}
[-16.37171961 -65.38920929 -63.43537025 -22.04081334 -43.13176901]
ridge cv error:  42.0737763010807


### (c) features

In [34]:
#lasso기준 선택
lasso4= Lasso(lass2.alpha_)
print("lasso alpha: ", lass2.alpha_)
X_scaled = StandardScaler().fit_transform(X) 
lasso4.fit(X_scaled, y)
for i in range(X.shape[1]):
    print('\"'+X.columns[i]+'\"', ' coefficient: ', lasso4.coef_[i])
    
print("사용된 feature수 : 10개, \n제외된 feature 2개 : ", X.columns[5], "and", X.columns[8])
print("lasso score: ", lasso4.score(StandardScaler().fit_transform(X),y))

lasso alpha:  0.05374991578689812
"zn"  coefficient:  0.8667056976899319
"indus"  coefficient:  -0.43847712781981374
"chas"  coefficient:  -0.1687793792155293
"nox"  coefficient:  -0.7673815711401292
"rm"  coefficient:  0.28333209090928274
"age"  coefficient:  -0.0
"dis"  coefficient:  -1.670814196505264
"rad"  coefficient:  4.683328397027171
"tax"  coefficient:  -0.0
"ptratio"  coefficient:  -0.47379862713398746
"lstat"  coefficient:  0.955857754502836
"medv"  coefficient:  -1.6334482204774372
사용된 feature수 : 10개, 
제외된 feature 2개 :  age and tax
lasso score:  0.4472329074404988
